# 1. Embedding Model (向量模型)
在介紹向量存儲前 (Vectorstore)，先來了解什麼是向量 (embedding) 和如何製作向量。

透過文字向量模型，可以將一段文字 (text) 轉成一串數字，也就是向量。注意每個向量長度，也就是有幾個數字，是固定的不依文字長短改變。

這個文字向量可以代表這段文字的語義 (可參考)。

這些向量有很多不同的用途，但在這邊我們是為了做以向量為基礎的文件抽取 (embedding-based retrieval)。利用相似的文字會有相似的向量的特性，透過比較文字向量間的相似度來找出相似的文件。

In [1]:
import numpy as np
from numpy.linalg import norm
from langchain.embeddings import OpenAIEmbeddings
import langchain_setup

embedding_model = OpenAIEmbeddings()  # 文字向量模型, 預設為 text-embedding-ada-002

透過向量模型 (embedding model)，我們會得到一連串的數字代表每段文字 (text)

In [2]:
# 將一堆文字 (texts) 轉成向量 (embeddings)
texts = [
    "英國最新研究顯示：人被殺就會死。",  # document 1
    "海的對面有什麼，海的對面有敵人。",  # document 2
    "努估誰唷？",  # document 3
    "一袋米要扛幾樓？一袋米要扛二樓。",  # document 4
]
document_embeddings: list[list[float]] = embedding_model.embed_documents(texts)
document_embeddings = np.array(document_embeddings)
print("document_embeddings:", document_embeddings.shape)

# 將一段文字 (text) 轉成向量 (embedding)
query = "扛一袋米有多麻煩？"
query_embeddings: list[float] = embedding_model.embed_query(query)
query_embeddings = np.array(query_embeddings)
print("query_embeddings:", query_embeddings.shape)

document_embeddings: (4, 1536)
query_embeddings: (1536,)


透過比較向量 (embedding) 我們可以看出誰跟誰更相似。(之後介紹的向量存儲 (vectorstore) 會自動幫我們做這些事，所以不必太在意看不懂程式碼)，這邊用的是cosine 相似度，介於-1~1之間，越高越像。

In [3]:
def cosine_similarity(a, b):
    return (a @ b.T) / (norm(a) * norm(b))


similarities = cosine_similarity(query_embeddings[None, :], document_embeddings)
for text, similarity in zip(texts, similarities[0]):
    print(f"相似度({query}, {text}): {similarity}")

相似度(扛一袋米有多麻煩？, 英國最新研究顯示：人被殺就會死。): 0.3709144687064924
相似度(扛一袋米有多麻煩？, 海的對面有什麼，海的對面有敵人。): 0.388389953971131
相似度(扛一袋米有多麻煩？, 努估誰唷？): 0.4101972948317287
相似度(扛一袋米有多麻煩？, 一袋米要扛幾樓？一袋米要扛二樓。): 0.45302219992860476


**想想看：**
1. 相似就一定相關嗎？
2. 當文章非常長，但向量的長度又不會變，在壓縮的過程中是否為丟失資訊？
3. 語義 (semantics) 相似是否可以取代語面 (lexical) 相似

# 2. Vector Store 介紹

## 2.1 什麼是 Vectorstore?
Vector Store 主要提供以下的功能：
- 儲存文件 (document) 和對應的embedding (storing embedded data)
- 藉由比較 embedding 的相似度，抽取出對應的文件 (performing vector search)

## 2.2 要用哪種 Vectorstore?
我個人的挑選過程如下

1. 從 [Langchain - Vector stores](https://python.langchain.com/docs/integrations/vectorstores/) 中找到目前 Langchain 有支援的作為候選

2. 能夠不用另外開 server（Server Storage）或擾亂檔案系統 (Disk Storage) ，使用程式關閉後就會消失的存儲 (In-memory storage)，對初期或原型的開發有著非常重要的好處。執筆時只知道 Chroma 和 Qdrant 有這個功能。

    - 但 Chroma 提供的是假 In-memory storage，它會存檔案在 `~/.chroma` 裡，等程式結束後再刪掉。所以在同一程式執行中開啟的 Chroma 都會連到同一個資料庫 (除非手動指定 collection)。
        ```
        c1 = Chroma.from_documents(documents=[docs[0]], embedding=OpenAIEmbeddings())
        c2 = Chroma.from_documents(documents=[docs[1]], embedding=OpenAIEmbeddings())
        print(c1.get()['documents'])
        print(c2.get()['documents'] )
        # c1 和 c2 都會包含 docs[0] 和 docs[1]，明明是建立兩個資料庫卻會累積 
        ```

3. 表現 (Performance)：Qdrant 做過一個評測 (Benchmark) 來證明它們的速度很快準度很好。[連結](https://qdrant.tech/benchmarks/)。另外 Qdrant 是以 Rust 撰寫， Chroma 似乎是利用 Python 呼叫 C++ 程式庫。

4. 功能 (Features)：Qdrant 是少數支援 async 的，Chroma 則尚未支援

5. 教學文件 (Documentation)：Chroma 的教學文件網站沒有搜尋引擎

6. 未來成長性：很難評估，但其中一種方式是透過獲得的融資判斷。Chroma 獲得 1800 萬融資，Qdrant 獲得 750 萬融資。

7. License：Qdrant 跟 Chroma 都是開源可商用

根據執筆時得到的資料和判斷，以下介紹 Qdrant 的用法，並且作為接下來的內容使用的 Vectorstore。

# 3. Vector store 使用方法

In [1]:
import os
from pprint import pprint
from qdrant_client import QdrantClient
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Qdrant
from langchain.schema import Document
from langchain.indexes import SQLRecordManager, index

# 我寫的輔助函式 (helper functions)
from langchain_setup.qdrant import create_empty_qdrant, pprint_qdrant_documents

## 3.1 Creation (建立)
Langchain 的 vectorstore 介面 (interface) 為所有廠牌的 vectorstore 提供了兩種方式，`from_texts` 和 `from_documents`，來建立新的 vectorstore。

而建立 Qdrant 牌的 vectorstore 時，不能直接建立一個空的 vectorstore。所以我寫了一個 `create_empty_qdrant` 的小函式來方便做到這件事。（建立 Qdrant 需要提供 embedding 向量的長度，而這個需要先傳文件進去 embed 後才能得到，所以先建立一個只有一個字的文件的 vectorstore 後再把該文件刪掉來做出空的 qdrant vectorstore)

In [2]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Qdrant
from langchain.schema import Document
from langchain_setup.qdrant import create_empty_qdrant, pprint_qdrant_documents # # 我寫的輔助函式 (helper functions)

print("建立一個空的 Qdrant Vectorstore", end="\n\n")
empty_qdrant = create_empty_qdrant(
    embedding=OpenAIEmbeddings(),  # embedding model
    # Qdrant 特有 (specific) 參數 (key word arguments)
    location=":memory:",  # 存在 RAM 裡
)
pprint_qdrant_documents(empty_qdrant)
print("=======================================")

print("從文字自動建立文件和 Vectorstore。 document metadata 為空", end="\n\n")
qdrant_from_texts = Qdrant.from_texts(
    texts=[
        "一言不合，麻美掉頭就走",
        "妮娜小妹妹的大哥哥",
    ],
    embedding=OpenAIEmbeddings(),  # embedding model
    location=":memory:",
)
pprint_qdrant_documents(qdrant_from_texts)
print("=======================================")

print("建立包含這些文件的新 Vectorstore", end="\n\n")
qdrant_from_docs = Qdrant.from_documents(
    documents=[
        Document(page_content="一言不合，麻美掉頭就走", metadata={"種族": "durahan"}),
        Document(page_content="妮娜小妹妹的大哥哥", metadata={"物種": "chimera"}),
    ],
    embedding=OpenAIEmbeddings(),  # embedding model
    location=":memory:",
)
pprint_qdrant_documents(qdrant_from_docs)
print("=======================================")

建立一個空的 Qdrant Vectorstore


從文字自動建立文件和 Vectorstore。 document metadata 為空

Document 02b88eed97c24b1e97123ef47fda4570:

妮娜小妹妹的大哥哥

Metadata:{}
----------------------------------------------------------------------------------------------------
Document c67abe4b30544dcfb0a4da9018fd210c:

一言不合，麻美掉頭就走

Metadata:{}
建立包含這些文件的新 Vectorstore

Document b89546979e1b43e8a5d7ed38b1a1d18a:

妮娜小妹妹的大哥哥

Metadata:{'物種': 'chimera'}
----------------------------------------------------------------------------------------------------
Document cf061bd03209444bb4b25ead679d07f7:

一言不合，麻美掉頭就走

Metadata:{'種族': 'durahan'}


## 3.2 Collections
每一個 Collection（某些廠牌叫 index）是一群資料的集合，collections 實現了資料的分隔 (partition)，在某個 collection 做搜尋時不會搜尋到其他的 collection。但注意關於是否要用多個 collection 可以參考[官方的文章](https://qdrant.tech/documentation/concepts/collections/)

In [1]:
from langchain.embeddings import OpenAIEmbeddings
from langchain_setup.qdrant import create_empty_qdrant

empty_qdrant = create_empty_qdrant(
    embedding=OpenAIEmbeddings(),  # embedding model
    # Qdrant 特有 (specific) 參數 (key word arguments)
    location=":memory:",  # 存在 RAM
    collection_name="empty_empty_like_and",  # 空空如也
)

## 3.3 Storage (存儲)
每家廠牌支援的存儲類型，使用方式，實作方式都不一樣，這邊以 Qdrant 介紹。

這邊介紹兩種地端的存儲方式，雲端方式可以參考[官方的文章](https://qdrant.tech/documentation/integrations/langchain/)

### 3.3.1 In-memory
存在 memory 中，程式結束或實體 (instance) 消滅後就會消滅

In [1]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Qdrant
import langchain_setup

qdrant_from_texts = Qdrant.from_texts(
    texts=[
        "一言不合，麻美掉頭就走",
        "妮娜小妹妹的大哥哥",
    ],
    embedding=OpenAIEmbeddings(),  # embedding model
    location=":memory:",
)

### 3.3.2 Disk
存在地端檔案系統中

In [1]:
import os
from pprint import pprint
from tempfile import TemporaryDirectory
from qdrant_client import QdrantClient
from langchain.schema import Document
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Qdrant
from langchain_setup.qdrant import pprint_qdrant_documents

tmp_dir = TemporaryDirectory()

qdrant_from_texts = Qdrant.from_texts(
    texts=[
        "一言不合，麻美掉頭就走",
        "妮娜小妹妹的大哥哥",
    ],
    embedding=OpenAIEmbeddings(),  # embedding model
    path=tmp_dir.name,
    collection_name="disk_test",
)

def list_files(startpath):
    for root, dirs, files in os.walk(startpath):
        level = root.replace(startpath, "").count(os.sep)
        indent = " " * 4 * (level)
        print("{}{}/".format(indent, os.path.basename(root)))
        subindent = " " * 4 * (level + 1)
        for f in files:
            print("{}{}".format(subindent, f))


list_files(tmp_dir.name)

tmp49ljeofg/
    .lock
    meta.json
    collection/
        disk_test/
            storage.sqlite


## 3.4 使用既存的資料庫
以 disk 存儲為例

In [2]:
qdrant_from_texts.client.close()  # 開啟同一個硬碟路徑的 client 只能有一個，為示範建立 client，把前面的先關掉

In [3]:
vectorstore = Qdrant(
    client=QdrantClient(path=tmp_dir.name),
    collection_name="disk_test",  # 必須
    embeddings=OpenAIEmbeddings(),  # 必須跟先前用的是同一個 embedding model
)
pprint_qdrant_documents(vectorstore)

Document 8d9d139b9c154229a8e725087967a324:

一言不合，麻美掉頭就走

Metadata:{}
----------------------------------------------------------------------------------------------------
Document 95d2adf5f31144c2bfc563c5889c9026:

妮娜小妹妹的大哥哥

Metadata:{}


## 3.5 檢視內部
這個得直接使用各個廠牌自己的函式庫 (Library) 的實作。

而我寫的 `pprint_qdrant_documents` 也是基於 Qdrant 所提供的 `scroll` 函式來實作的。

In [4]:
print("接下來使用的是 Qdrant 而非 Langchain 的功能")
print(vectorstore.client, end="\n\n")

print("檢視有哪些 collections: ")
print(vectorstore.client.get_collections(), end="\n\n")

print("檢視有哪些文件：")
# 顯示前十個紀錄 (Record)
records = vectorstore.client.scroll(collection_name=vectorstore.collection_name, limit=10)[0]
pprint(records)

接下來使用的是 Qdrant 而非 Langchain 的功能

檢視有哪些 collections: 
collections=[CollectionDescription(name='disk_test')]

檢視有哪些文件：
[Record(id='8d9d139b9c154229a8e725087967a324', payload={'page_content': '一言不合，麻美掉頭就走', 'metadata': None}, vector=None),
 Record(id='95d2adf5f31144c2bfc563c5889c9026', payload={'page_content': '妮娜小妹妹的大哥哥', 'metadata': None}, vector=None)]


## 3.6 Insert (新增) / Delete (移除)

In [5]:
vectorstore.delete([records[0].id])
pprint_qdrant_documents(vectorstore)

Document 95d2adf5f31144c2bfc563c5889c9026:

妮娜小妹妹的大哥哥

Metadata:{}


In [6]:
vectorstore.add_documents([Document(page_content="講到一半，麻理茉教官掉頭就走")])
pprint_qdrant_documents(vectorstore)

Document 5b8c1e3274d24e50ab551d62c525a981:

講到一半，麻理茉教官掉頭就走

Metadata:{}
----------------------------------------------------------------------------------------------------
Document 95d2adf5f31144c2bfc563c5889c9026:

妮娜小妹妹的大哥哥

Metadata:{}


## 3.7 Index = Insert (新增) + Delete (移除) + Update (更新)
每次都要檢查內部的識別碼 (id) 或文件內容 (page_content) 來的新增文件或決定哪個要刪掉，是一件非常麻煩的事。

Langchain 提供了`index`功能，告訴它想要的所有文件，它會自動透過新增和移除來更新向量存儲 (vectorstore)。其有三種不同的更新邏輯 `None`、`incremental`、`full`，以下介紹。

目前 Langchain 的程式碼裡有個 bug，我還沒ＰＲ，下面是 bug出處和解法：

C:\Users\121664\micromamba\envs\dev\Lib\site-packages\langchain\indexes\_api.py:285
```
for hashed_doc, doc_exists in zip(hashed_docs, exists_batch):
    if doc_exists:
        # Must be updated to refresh timestamp.
        record_manager.update([hashed_doc.uid], time_at_least=index_start_dt)
        num_skipped += 1
        continue
    uids.append(hashed_doc.uid)
    docs_to_index.append(Document(page_content=hashed_doc.page_content, metadata=hashed_doc.metadata))
```

In [1]:
from tempfile import TemporaryDirectory
from pathlib import Path
from pprint import pprint
from langchain.indexes import SQLRecordManager, index
from langchain.schema import Document
from langchain.vectorstores import Qdrant
from langchain_setup.qdrant import pprint_qdrant_documents, create_inmemory_empty_qdrant

In [2]:
# 文件 (Documents)
## 第一群
metadata1 = {"source": "fullhell.alchemist"}
doc1_1 = Document(page_content="1-1 I have a dog~", metadata=metadata1)
doc1_2 = Document(page_content="1-2 I have a daugter~", metadata=metadata1)
doc1_3 = Document(page_content="1-3 Ahh! O..Oniichan", metadata=metadata1)
## 第二群
doc2 = Document(page_content="2 Lancer 你又死了", metadata={"source": "fate.docx"})
## 第三群 (只要 source 一樣就算)
doc3_1 = Document(page_content="3-1 小笨狗", metadata={"source": "doggy.txt", "id": 0})
doc3_2 = Document(page_content="3-2 你害我好丟臉", metadata={"source": "doggy.txt"})

# 建立空的向量存儲 (vectorstore)
collection_name = "secret_of_D_disk"
vectorstore: Qdrant = create_inmemory_empty_qdrant()

# 建立空的紀錄管理器 (Record Manager)，其負責是否為同一文件 (document) 的識別
tmp_dir = TemporaryDirectory()
record_manager = SQLRecordManager(
    namespace="qdrant/{collection_name}",
    db_url=f"sqlite:///{Path(tmp_dir.name)/collection_name}.sql",
)
record_manager.create_schema()  # 必須

### 3.7.1 `Full` 模式

確保整個 vectorstore 都跟輸入文件 (documents)一模一樣，是邏輯最簡單的模式。

另外新增重複的文件會被跳過，避免了保存重複文件和重複 embedding 計算。

In [3]:
# 原本沒有任何文件
sync_result = index(
    [doc1_1, doc1_2, doc1_2, doc2],
    record_manager,
    vectorstore,
    cleanup="full",
    source_id_key="source",
)
print(sync_result, end="\n\n")
pprint_qdrant_documents(vectorstore)

{'num_added': 3, 'num_updated': 0, 'num_skipped': 0, 'num_deleted': 0}

Document 1b19816e-b802-53c0-ad60-5ff9d9b9b911:

1-2 I have a daugter~

Metadata:{'source': 'fullhell.alchemist'}
----------------------------------------------------------------------------------------------------
Document 3362f9bc-991a-5dd5-b465-c564786ce19c:

1-1 I have a dog~

Metadata:{'source': 'fullhell.alchemist'}
----------------------------------------------------------------------------------------------------
Document a4d50169-2fda-5339-a196-249b5f54a0de:

2 Lancer 你又死了

Metadata:{'source': 'fate.docx'}


In [4]:
# 原本包含 1-1, 1-2, 2
sync_result = index(
    [doc1_3, doc3_1, doc3_2],
    record_manager,
    vectorstore,
    cleanup="full",
    source_id_key="source",
)
print(sync_result, end="\n\n")
pprint_qdrant_documents(vectorstore)

{'num_added': 3, 'num_updated': 0, 'num_skipped': 0, 'num_deleted': 3}

Document 3b001c6e-9f06-5e9c-ba55-cfbacae830db:

3-1 小笨狗

Metadata:{'id': 0, 'source': 'doggy.txt'}
----------------------------------------------------------------------------------------------------
Document 8b035207-8426-579d-b190-f48d5fc578d3:

3-2 你害我好丟臉

Metadata:{'source': 'doggy.txt'}
----------------------------------------------------------------------------------------------------
Document f4bb822a-5594-543b-94a4-f2c1d0390c1a:

1-3 Ahh! O..Oniichan

Metadata:{'source': 'fullhell.alchemist'}


實際上是否要新增或刪除、判別文件是否已存在的任務是由 `record_manager` 執行的。（所以若將 record_manager 使用的 SQL 檔案刪掉後重建，但向量存儲 (vectorstore) 沒有清空的話，還是可能會在向量存儲裡儲存重複的文件）

而他而它利用文件 (Document) 的內容 (page_content) 和詮釋資料 (metadata) 來做 hash，並以該 hash 作為該文件的識別碼 (uid) 並以此判定是否為同一文件。

**注意:**

vectorstore (由各廠牌提供) 和 record manager (由 Langhcain 提供) 兩者是獨立的，使用者必須自己小心讓兩者同步:
- 建立時兩者皆必須為空
- 之後都不得用 index 以外的方式來更新任何一方

In [5]:
pprint(record_manager.list_keys())

['f4bb822a-5594-543b-94a4-f2c1d0390c1a',
 '3b001c6e-9f06-5e9c-ba55-cfbacae830db',
 '8b035207-8426-579d-b190-f48d5fc578d3']


### 3.7.2 `Incremental` 模式

刪除的時候，都只會針對跟輸入文件 (documents) 有相同的來源 (source) 的文件，其他文件保持不動。也就是只有跟任一輸入文件有相同來源文件的地方會跟輸入文件一模一樣。

另外新增重複的文件會被跳過，避免了保存重複文件和重複 embedding 計算。

In [6]:
# 原本包含 1-3, 3-1, 3-2
sync_result = index(
    [doc1_1, doc1_1, doc1_3, doc2],
    record_manager,
    vectorstore,
    cleanup="incremental",
    source_id_key="source",
)
print(sync_result, end="\n\n")
pprint_qdrant_documents(vectorstore)

{'num_added': 2, 'num_updated': 0, 'num_skipped': 1, 'num_deleted': 0}

Document 3362f9bc-991a-5dd5-b465-c564786ce19c:

1-1 I have a dog~

Metadata:{'source': 'fullhell.alchemist'}
----------------------------------------------------------------------------------------------------
Document 3b001c6e-9f06-5e9c-ba55-cfbacae830db:

3-1 小笨狗

Metadata:{'id': 0, 'source': 'doggy.txt'}
----------------------------------------------------------------------------------------------------
Document 8b035207-8426-579d-b190-f48d5fc578d3:

3-2 你害我好丟臉

Metadata:{'source': 'doggy.txt'}
----------------------------------------------------------------------------------------------------
Document a4d50169-2fda-5339-a196-249b5f54a0de:

2 Lancer 你又死了

Metadata:{'source': 'fate.docx'}
----------------------------------------------------------------------------------------------------
Document f4bb822a-5594-543b-94a4-f2c1d0390c1a:

1-3 Ahh! O..Oniichan

Metadata:{'source': 'fullhell.alchemist'}


### 3.7.3 `None` 模式
在 `None` 模式下，任何情況都不會刪除文件。也就是只做輸入的文件 (documents) 在向量存儲 (vectorstore) 裡沒有的話就新增這件事

In [7]:
# 原本包含 1-1, 1-3, 2, 3-1, 3-2
sync_result = index(
    [doc1_1, doc1_2, doc1_2],
    record_manager,
    vectorstore,
    cleanup=None,
    source_id_key="source",
)
print(sync_result, end='\n\n')
pprint_qdrant_documents(vectorstore)

{'num_added': 1, 'num_updated': 0, 'num_skipped': 1, 'num_deleted': 0}

Document 1b19816e-b802-53c0-ad60-5ff9d9b9b911:

1-2 I have a daugter~

Metadata:{'source': 'fullhell.alchemist'}
----------------------------------------------------------------------------------------------------
Document 3362f9bc-991a-5dd5-b465-c564786ce19c:

1-1 I have a dog~

Metadata:{'source': 'fullhell.alchemist'}
----------------------------------------------------------------------------------------------------
Document 3b001c6e-9f06-5e9c-ba55-cfbacae830db:

3-1 小笨狗

Metadata:{'id': 0, 'source': 'doggy.txt'}
----------------------------------------------------------------------------------------------------
Document 8b035207-8426-579d-b190-f48d5fc578d3:

3-2 你害我好丟臉

Metadata:{'source': 'doggy.txt'}
----------------------------------------------------------------------------------------------------
Document a4d50169-2fda-5339-a196-249b5f54a0de:

2 Lancer 你又死了

Metadata:{'source': 'fate.docx'}
--------------

**想想看**: 這三個模式分別適合在什麼情況下使用？